In [1]:
import math
from datetime import datetime

import pymysql

In [2]:
property = {
    "host": "",
    "user": "",
    "password": "",
    "database": "",
}

In [3]:
def sql(connection, query, method = "all"):
    with connection.cursor() as cursor:
        cursor.execute(query)
        if method == "all":
            result = cursor.fetchall()
        elif method == "one":
            result = cursor.fetchone()
        else:
            ValueError("query error:", query)

    return result

def get_connection():
    return pymysql.connect(
        host=property["host"],
        user=property["user"],
        password=property["password"],
        database=property["database"],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor,
    )

In [4]:
dms_ids_c = {
    "A": 7,
    "B": 7,
    "C": 7,
    "I": 8,
    "L": 8,
    "R": 9,
    "S": 7,
}
dms_p_count = 28

z_fields = [f"z_{prefix}{str(num).zfill(2)}" for prefix, max_num in dms_ids_c.items() for num in range(1, max_num + 1)]
z_fields += [f"z_D{str(num).zfill(2)}" for num in range(1, dms_p_count + 1)]
z_fields_str = ", ".join(z_fields)
print(z_fields_str)
print()

sample_fields = f"tgu_id, idno, cid, invoice, industry_id, position_id, sm, sy, finish_date, {z_fields_str}".split(", ")
print(sample_fields)

z_A01, z_A02, z_A03, z_A04, z_A05, z_A06, z_A07, z_B01, z_B02, z_B03, z_B04, z_B05, z_B06, z_B07, z_C01, z_C02, z_C03, z_C04, z_C05, z_C06, z_C07, z_I01, z_I02, z_I03, z_I04, z_I05, z_I06, z_I07, z_I08, z_L01, z_L02, z_L03, z_L04, z_L05, z_L06, z_L07, z_L08, z_R01, z_R02, z_R03, z_R04, z_R05, z_R06, z_R07, z_R08, z_R09, z_S01, z_S02, z_S03, z_S04, z_S05, z_S06, z_S07, z_D01, z_D02, z_D03, z_D04, z_D05, z_D06, z_D07, z_D08, z_D09, z_D10, z_D11, z_D12, z_D13, z_D14, z_D15, z_D16, z_D17, z_D18, z_D19, z_D20, z_D21, z_D22, z_D23, z_D24, z_D25, z_D26, z_D27, z_D28

['tgu_id', 'idno', 'cid', 'invoice', 'industry_id', 'position_id', 'sm', 'sy', 'finish_date', 'z_A01', 'z_A02', 'z_A03', 'z_A04', 'z_A05', 'z_A06', 'z_A07', 'z_B01', 'z_B02', 'z_B03', 'z_B04', 'z_B05', 'z_B06', 'z_B07', 'z_C01', 'z_C02', 'z_C03', 'z_C04', 'z_C05', 'z_C06', 'z_C07', 'z_I01', 'z_I02', 'z_I03', 'z_I04', 'z_I05', 'z_I06', 'z_I07', 'z_I08', 'z_L01', 'z_L02', 'z_L03', 'z_L04', 'z_L05', 'z_L06', 'z_L07', 'z_L08', 'z_R01

In [5]:
connection = get_connection()

query = """
    SELECT DISTINCT idno
    FROM target_user_sample
    WHERE idno > 0
    ORDER BY idno
    LIMIT 10000
"""
result = sql(connection, query)
idnos = [row["idno"] for row in result]
print(len(idnos))

if connection.open:
    connection.close()
print(connection.open)

10000
False


In [6]:
tgu_str = f"tgu_id, idno, finish_date, {z_fields_str}"
tgu_str

'tgu_id, idno, finish_date, z_A01, z_A02, z_A03, z_A04, z_A05, z_A06, z_A07, z_B01, z_B02, z_B03, z_B04, z_B05, z_B06, z_B07, z_C01, z_C02, z_C03, z_C04, z_C05, z_C06, z_C07, z_I01, z_I02, z_I03, z_I04, z_I05, z_I06, z_I07, z_I08, z_L01, z_L02, z_L03, z_L04, z_L05, z_L06, z_L07, z_L08, z_R01, z_R02, z_R03, z_R04, z_R05, z_R06, z_R07, z_R08, z_R09, z_S01, z_S02, z_S03, z_S04, z_S05, z_S06, z_S07, z_D01, z_D02, z_D03, z_D04, z_D05, z_D06, z_D07, z_D08, z_D09, z_D10, z_D11, z_D12, z_D13, z_D14, z_D15, z_D16, z_D17, z_D18, z_D19, z_D20, z_D21, z_D22, z_D23, z_D24, z_D25, z_D26, z_D27, z_D28'

In [7]:
import multiprocessing as mp

def process(idnos, tgu_str):
    connection = get_connection()
    start = datetime.now()
    with connection:
        sample_datas = []
        for i, idno in enumerate(idnos, start=1):
            query = f"""
                SELECT {tgu_str}
                FROM target_user_sample
                WHERE idno = {idno}
            """
            tgus = sql(connection, query)
            if len(tgus) == 0:
                continue

            query = f"""
                SELECT invoice, ind_cat_no, job_cat_no, start_date, end_date
                FROM exp_job
                WHERE id_no = {idno} AND (job_cat_no > 0 OR ind_cat_no > 0)
            """
            exps = sql(connection, query)
            if len(exps) == 0:
                continue

            for exp in exps:
                start_ts = exp["start_date"].timestamp()
                start_y = int(exp["start_date"].strftime("%Y"))
                start_m = int(exp["start_date"].strftime("%m"))

                if exp["end_date"] and start_ts < exp["end_date"].timestamp():
                    end_ts = exp["end_date"].timestamp()
                    end_y = int(exp["end_date"].strftime("%Y"))
                    end_m = int(exp["end_date"].strftime("%m"))
                else:
                    if not exp["end_date"]:
                        print(f"~~~ -> {exp['end_date']}")

                    query = f"SELECT update_date FROM basic WHERE id_no = {idno}"
                    basic = sql(connection, query, "one")
                    end_ts = basic["update_date"].timestamp()
                    end_y = int(basic["update_date"].strftime("%Y"))
                    end_m = int(basic["update_date"].strftime("%m"))

                sm = (end_y - start_y) * 12 + (end_m - start_m)
                sm = 750 if sm > 750 else sm

                sy = -9.9999999999
                if exp["end_date"] and start_ts < exp["end_date"].timestamp():
                    if sm == 0:
                        sy = round(2 + math.log2(0.5 / 3), 10)
                    else:
                        sy = round(2 + math.log2(sm / 3), 10)

                min_diff = 1e20
                for tgu in tgus:
                    cur_diff = abs((start_ts + end_ts) / 2 - tgu["finish_date"].timestamp())
                    if cur_diff < min_diff:
                        target_tgu = tgu
                        min_diff = cur_diff

                query = f"""SELECT cid FROM cid_mapping WHERE invoice = {exp['invoice']} ORDER BY cid"""
                cid = 0 if int(exp["invoice"]) == 0 else sql(connection, query, "one")
                cid = cid if cid else {"cid": 0}
                sample_data = target_tgu.copy()
                sample_data["cid"] = cid["cid"]
                sample_data["invoice"] = int(exp["invoice"])
                sample_data["industry_id"] = exp["ind_cat_no"]
                sample_data["position_id"] = exp["job_cat_no"]
                sample_data["sm"] = sm
                sample_data["sy"] = sy
                sample_datas.append(sample_data)

            if i % 2000 == 0 or i == len(idnos):
                end = datetime.now()
                time_taken = end - start
                print(f"{mp.current_process().name}, {i} completed in {time_taken}")
    
    if connection.open:
        connection.close()
    print(len(sample_datas))
    
    return sample_datas

In [8]:
from concurrent.futures import ProcessPoolExecutor

start = datetime.now()

max_workers = 8
executor = ProcessPoolExecutor(max_workers)
batch_size = math.ceil(len(idnos) / max_workers)

futures = [
    executor.submit(
        process,
        idnos[i * batch_size : (i + 1) * batch_size],
        tgu_str,
    )
    for i in range(max_workers)
]

sample_datas = []
for future in futures:
    sample_datas.extend(future.result())

print(len(sample_datas))
end = datetime.now()
time_taken = end - start
# cid_mapping 的 invoice 欄位有加 index
print(f"Multi-processing completed in {time_taken}")

4547
ForkProcess-2, 1250 completed in 0:00:16.658529
4993
ForkProcess-8, 1250 completed in 0:00:18.754067
4688
ForkProcess-7, 1250 completed in 0:00:19.524154
5143
ForkProcess-3, 1250 completed in 0:00:19.847261
4794
ForkProcess-4, 1250 completed in 0:00:19.886880
4611
ForkProcess-6, 1250 completed in 0:00:19.931444
4814
ForkProcess-5, 1250 completed in 0:00:22.374604
4742
38332
Multi-processing completed in 0:00:22.536844
